In [1]:
import os

In [2]:
%pwd

'c:\\Users\\SIR\\Credit-Card-Default-Prediction-Project-Pwskills\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\SIR\\Credit-Card-Default-Prediction-Project-Pwskills'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    

In [7]:
from ccdp.constants import *
from ccdp.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [9]:
#src/textSummerizer/components/data_ingestion.py
import os
import urllib.request as request
import zipfile
from ccdp.logging import logger
from ccdp.utils.common import get_size

In [10]:
#src/textSummerizer/components/data_ingestion.py
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [12]:
#src/textSummerizer/pipeline/stage_01_data_ingestion.py
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-08-16 01:36:30,071: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-16 01:36:30,093: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-16 01:36:30,096: INFO: common: created directory at: artifacts]
[2024-08-16 01:36:30,100: INFO: common: created directory at: artifacts/data_ingestion]
[2024-08-16 01:36:36,791: INFO: 1250927452: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 995594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "a5dc476325f73cca3c9107b7c4fcc8ada5cc22d37e45225c959dae6c8f0828e2"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: E3FF:2C2CA1:9E697:BEE8C:66BE5FC5
Accept-Ranges: bytes
Date: Thu, 15 Aug 2024 20:06:35 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4727-BOM
X-Cache: MISS
